In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_sentiment_model')
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_sentiment_model')

# Set the model to evaluation mode
model.eval()

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [2]:
###DOES NOT WORK###

'''
def classifier(texts):
    # Tokenize input texts
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Perform a forward pass through the model and get logits
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

    # Create a list of labels
    labels = ["NEGATIVE", "POSITIVE"]

    # Format the output in the same structure as the pipeline
    results = []
    for prob in probabilities:
        results.append([{ "label": labels[i], "score": float(prob[i]) } for i in range(len(labels))])

    return results
texts = ["This movie was great!", "I did not enjoy the film at all."]
results = classifier(texts)
print(results)
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(classifier, texts)

# Generate SHAP values for the first two texts
shap_values = explainer(texts[:2])

# Visualize SHAP explanations for the "POSITIVE" label (index 1)
shap.plots.text(shap_values[:, :, 1])
'''

'\ndef classifier(texts):\n    # Tokenize input texts\n    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")\n\n    # Perform a forward pass through the model and get logits\n    with torch.no_grad():\n        outputs = model(**inputs)\n\n    # Convert logits to probabilities\n    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()\n\n    # Create a list of labels\n    labels = ["NEGATIVE", "POSITIVE"]\n\n    # Format the output in the same structure as the pipeline\n    results = []\n    for prob in probabilities:\n        results.append([{ "label": labels[i], "score": float(prob[i]) } for i in range(len(labels))])\n\n    return results\ntexts = ["This movie was great!", "I did not enjoy the film at all."]\nresults = classifier(texts)\nprint(results)\nimport shap\n\n# Initialize the SHAP explainer\nexplainer = shap.Explainer(classifier, texts)\n\n# Generate SHAP values for the first two texts\nshap_values = expl

In [3]:
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_83')
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_83')

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

texts = ["This movie was great!", "I did not enjoy the film at all."]
classifier(texts)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/home/j597s263/NLP_DP/SHAP/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 0.003580017015337944},
  {'label': 'LABEL_1', 'score': 0.9964200258255005}],
 [{'label': 'LABEL_0', 'score': 0.9804778695106506},
  {'label': 'LABEL_1', 'score': 0.019522150978446007}]]

In [4]:
import datasets
dataset = datasets.load_dataset("imdb", split="test")

In [5]:
import shap

# define the explainer
explainer = shap.Explainer(classifier)

# explain the predictions of the pipeline on the first two samples
shap_values = explainer(texts)

shap.plots.text(shap_values[:, :, "LABEL_0"])

  0%|          | 0/110 [00:00<?, ?it/s]

PartitionExplainer explainer: 3it [00:10, 10.07s/it]                                                                                                                      


In [6]:
# Define a single text to explain
single_text = ["This movie was great!"]

# Generate SHAP values for the single text
shap_values = explainer(single_text)


In [7]:
# Visualize SHAP explanation for the "NEGATIVE" class (assumed to be "LABEL_0")
shap.plots.text(shap_values[:, :, "LABEL_1"])


In [8]:
print(shap_values)

.values =
array([[[ 0.        ,  0.        ],
        [ 0.24779552, -0.24779554],
        [-0.1239901 ,  0.12399009],
        [ 0.24790813, -0.24790813],
        [-0.86797722,  0.86797724],
        [-0.49525169,  0.49525169],
        [ 0.        ,  0.        ]]])

.base_values =
array([[0.99509537, 0.00490468]])

.data =
(array(['', ' this', ' movie', ' was', ' great', ' !', ''], dtype=object),)


In [11]:
# Initialize a dictionary to store aggregated SHAP values for each index
shap_dict = {i: 0.0 for i in range(512)}  # Dictionary with 512 indices initialized to 0.0

# Loop through the SHAP values and add them to the respective index in the dictionary
for i, shap_value in enumerate(shap_values.values[0]):
    if i < 512:  # Ensure we don't exceed 512 indices
        shap_dict[i] += float(shap_value[0])  # Convert to float before adding for the first class (e.g., negative class)

In [12]:
 from datasets import load_dataset
import shap
from transformers import pipeline, BertTokenizer, BertForSequenceClassification

# Load your custom model and tokenizer
model = BertForSequenceClassification.from_pretrained('/home/j597s263/bert_83').to("cuda")
tokenizer = BertTokenizer.from_pretrained('/home/j597s263/bert_83')

# Initialize the sentiment-analysis pipeline with truncation and GPU support
classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=0,  # Use GPU
    truncation=True  # Enable truncation to limit input length to 512 tokens
)

# Load IMDb test dataset
dataset = load_dataset("imdb", split="test")

# Initialize the SHAP explainer
explainer = shap.Explainer(classifier)

# Initialize a dictionary to store aggregated SHAP values for each index
shap_dict = {i: 0.0 for i in range(512)}  # Dictionary with 512 indices initialized to 0.0

# Loop over each example in the dataset
for i, example in enumerate(dataset['text']):
    # Generate SHAP values for the current example
    shap_values = explainer([example])

    # Aggregate SHAP values for each index in the dictionary
    for idx, shap_value in enumerate(shap_values.values[0]):
        if idx < 512:  # Ensure we stay within 512 indices
            shap_dict[idx] += float(shap_value[0])  # Convert to float before adding

    # Optional: Print progress every 100 examples
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1} samples...")

print("Aggregated SHAP values:", shap_dict)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

KeyboardInterrupt: 